In [1]:
import numpy as np
%run similarity_features.ipynb

loaded 54283 word embeddings!


In [6]:
def count_morphological_features(word_file: str, segmentation_file: str, roots_reference: dict)->tuple[dict,dict]:
    # call generate segmentations function
    proposed = {}
    chosen = {}
    new_word = True 
    current = ""

    with open(segmentation_file, "r") as file:
        lines = file.readlines()
        for line in lines[1:]:
            root = None 
            
            if new_word:
                current= line.strip()
                new_word = False 
                
            if line.strip() =="":
                new_word = True 
                continue # skip empty lines 
                
            elif line.startswith("+"):
                morphemes = line.strip().split("+")

                for morpheme in morphemes:
                    if morpheme.islower():
                        root = morpheme
                # asumption is that root will always be present 
                # root[morphemens] = number of times they are appearing together 
                for morpheme in morphemes:
                    if root not in proposed:
                        proposed[root] = {}
                        if morpheme not in proposed[root]:
                            proposed[root][morpheme] = 0
                        proposed[root][morpheme] += 1 
                         
                                    
                    # check if the root is in confirmed in the roots_reference 
                    if current in current_reference and root == roots_reference[current]:
                        if root not in chosen:
                            chosen[root] = {}
                        if morpheme not in chosen[root]:
                            chosen[root][morpheme] = 0
                        chosen[root][morpheme] += 1 
    return chosen, proposed 

In [7]:
def compute_selection_ratios(proposed:dict, chosen:dict, Minf:float, Maxf:float)->dict:
    selection_ratios = {}

    for root in proposed:
        feature_score = []

        for morpheme in proposed[root]:
            proposed_count = proposed[root][morpheme]
            chosen_count = chosen[root].get(morpheme, 0)

            if proposed_count > 0:
                ratio = chosen_count / proposed_count 
                normalized = sigmoid(ration, Minf, Maxf)
                feature_score.append(normalized)

            if feature_score:
                selection_ratios[root]= {
                    "selection_mean": np.mean(feature_scores),
                    "selection_harmonic_mean": len(feature_scores)/ np.sum([1.0/ s for s in feature_scores if s !=0]),
                    "selection_geometric_mean": np.exp(np.mean(np.log(feature_scores))) if all (s > 0 for s in feature_scores) else 0.0
                    
                }
            else:
                selection_ratios[root] = {"selection_mean": 0.0, "selection_harmonic_mean": 0.0, "selection_geometric_mean": 0.0}
    return selection_ratios
                
    